In [71]:
# Import library yang digunakan

from textblob import TextBlob
import tweepy
import pandas as pd
import numpy as py
import re
import string
import nltk
# Import library stopword
from nltk.corpus import stopwords 
# Import library sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# Import library tokenize
from nltk.tokenize import TweetTokenizer
# Import library membagi data train dan data test
from sklearn.model_selection import train_test_split
# Import library TF IDF pada dokumen kita
from sklearn.feature_extraction.text import TfidfVectorizer
# Import library model klasifikasi SVM
from sklearn import svm
from sklearn.model_selection import cross_val_score
# Import library evaluation 
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, accuracy_score

In [11]:
# API Key Token Twitter

mykeys = open('../API_key/apikey.txt', 'r').read().splitlines()

api_key = mykeys[0]
api_key_secret = mykeys[1]
access_token = mykeys[2]
access_token_secret = mykeys[3]
auth_hendler = tweepy.OAuthHandler(api_key, api_key_secret)
auth_hendler.set_access_token(access_token, access_token_secret)

In [12]:
# Crawling Data

api = tweepy.API(auth_hendler)
# Kata kunci dan jumlah data
search_term = 'biznet'
tweet_amount = 1000
# Between two dates (just 10 day)
start_date = "2020-12-31"
end_date = "2020-11-30"
noretweets = '-filter:retweets'
# Tweepy Cursor
tweets = tweepy.Cursor(api.search, q=search_term + noretweets, lang='id', since=start_date).items()
# Pulling information from tweets iterable 
tweets_ = [[tweet.created_at,tweet.user.screen_name, tweet.user.location, tweet.text] for tweet in tweets]
# Make DataFrame for tweets after crawling
tweets_list = pd.DataFrame(data=tweets_, columns=['Date','Username','Location','Text'])
# Creation of dataframe from tweets list
tweets_df = pd.DataFrame(tweets_list)

In [13]:
tweets_df

,Date,Username,Location,Text
0,2021-01-04 15:48:35,bubblechaz,Pangkuan Ibu Pertiwi,trus nanti di hunian baru developer nya kerjas...
1,2021-01-04 15:11:10,damar_komodo,Jotunheim,@ptrsyhn_ @ianhugen Kalo ada myrep atau biznet...
2,2021-01-04 15:05:02,fairywoodpprink,,"@diahfanny Biznet so far oke sih, tapi TV kabe..."
3,2021-01-04 14:37:41,ilu555ion,nowhere,rk biznet rk indih0m3 kok podobae lemote
4,2021-01-04 14:17:00,AbyanKingdoms,"Jakarta, Indonesia",@prmthdy @FirstMediaCares Ganti Biznet. Yg per...
...,...,...,...,...
251,2020-12-31 01:03:08,zahrakhrn,"ÜT: -6.33144,106.85462",Tolong dong biznet daerah pasar rebo udah bisa...
252,2020-12-31 01:02:39,zahrakhrn,"ÜT: -6.33144,106.85462",Gue gatau isp lain selain indihome sama biznet...
253,2020-12-31 00:42:30,SRaydipta,,"@BiznetHome min mohon dibales dm nya, diinfo k..."
254,2020-12-31 00:32:46,SRaydipta,,@BiznetHome min restart sudah tunggu sejam sud...


In [47]:
# Convert data to csv

tweets_df.to_csv('../Dataset/data.csv', index = False, header = True)

In [69]:
# Open file csv

tweets = pd.read_csv('../Dataset/test.csv',encoding='latin-1')
tweets.drop(['Date','Username','Location'], axis=1, inplace=True)

In [70]:
tweets

,Text,Labels
0,@BiznetHome maaf biznet internet error lama ba...,negatif
1,"@.biznet kita baikan aja yuk, foto di twitter ...",negatif
2,"Udah nyobain indihome,mnc,biznet.\nPayah semua...",negatif
3,udahan yu gangguannya biznet besok terakhir ut...,negatif
4,parah bgt da biznet error mulu dr semalem,negatif
...,...,...
995,@praditautama Biznet nya cepetan dong,positif
996,"@TimpalBali Biznet, 360.500 up to 75mbps tapi ...",positif
997,@ichsanli Wkwk tapii rumah lu ke cover biznet ...,positif
998,@_mamitoo Bbrp temen gua pake biznet baguuus. ...,positif


In [67]:
#MAU DI AMBIL#MAU DI AMBIL#MAU DI AMBIL#MAU DI AMBIL#MAU DI AMBIL#MAU DI AMBIL#MAU DI AMBIL#MAU DI AMBIL#MAU DI AMBIL
# tweets.drop_duplicates(subset ="Text", keep = 'first', inplace = True)
# tweets

def hapuskatadobel(s):
    pattern = re.compile(r'(.)\1{1,}', re.DOTALL)
    return pattern

tweets['Text'] = tweets['Text'].apply(lambda x: hapuskatadobel(x))
tweets

,Text,Labels
0,Emang biznet biznet lebih aman dan nyaman. In...,positif
1,@BiznetHome Halo.. Biznet Jogja knp jadi serin...,negatif
2,Biznet home lama-lama gue kepret ya. Sebulan 3...,negatif
3,@_happymeals_ @FirstMediaCares Pake biznet aja...,positif
4,"@BiznetNetworks begini nih, lagi penting mau m...",NaN


In [6]:
#TESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTESTTEST

slankword = {'aga' : 'sedikit',
             'agak' : 'sedikit',
             'ancur' : 'hancur',
             'bs' : 'bisa',
             'bener' : 'benar',
             'bgt' : 'banget',
             'bngt' : 'banget',
             'cm' : 'cuma',
             'dr' : 'dari',
             'emang' : 'memang',
             'ga' : 'tidak',
             'gblk' : 'goblok',
             'gak' : 'tidak',
             'gmn' : 'gimana',
             'jd' : 'jadi',
             'jgn' : 'jangan',
             'kek' : 'seperti',
             'kyk' : 'seperti',
             'knp' : 'kenapa',
             'knapa' : 'kenapa',
             'km' : 'kamu',
             'lg' : 'lagi',
             'lgi' : 'lagi',
             'lu' : 'kamu',
             'lemot' : 'lambat',
             'napa' : 'kenapa',
             'nnton' : 'nonton',
             'nyobain' : 'coba',
             'pake' : 'pakai',
             'paraaahhh' : 'parah',
             'sad' : 'sedih',
             'sdh' : 'sudah',
             'sm' : 'sama',
             'sampe' : 'sampai',
             'skrng' : 'sekarang',
             'skali' : 'sekali',
             'tp' : 'tapi',
             'trs' : 'terus',
             'thanks' : 'terima kasih',
             'uda' : 'sudah',
             'udah' : 'sudah',
             'udahan' : 'sudah',
             'ujan' : 'hujan',
             'yg' : 'yang',
             'yaa' : 'ya'
            }

komentar = [] #deklarasi variabel komentar pada list
komentar = tweets['Text'].values.tolist() #masukan data kedalam list

#import library yang di butuhkan
from openpyxl import load_workbook #library untuk menampilkan dokumen
import pandas as pd #import pandas 
from nltk.tokenize import word_tokenize #import library nltk - tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import library sastrawi untuk

for index, text in enumerate(komentar):
    text = text.lower()
    splitlines = word_tokenize(text)
    for word in splitlines:
        if word in slankword:
            text = text.replace(word, str(slankword[word]));
            komentar[index] = text

tweets['Text'] = pd.DataFrame(komentar)
tweets

#ENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTESTENDTEST

,Text,Labels
0,memang biznet lebih aman dan nyaman. internet ...,positif
1,@biznethome halo.. biznet jogja kenapa jadi se...,negatif
2,Biznet home lama-lama gue kepret ya. Sebulan 3...,negatif
3,@_happymeals_ @firstmediacares pakai biznet aj...,positif
4,"@BiznetNetworks begini nih, lagi penting mau m...",NaN


In [26]:
# Proses Prerocessing

#import stopword
stopwords_indonesia = stopwords.words('indonesian')

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

slankword = { 
   'sad' : 'sedih',
    'thanks' : 'terima kasih',
    'bgt' : 'banget'
}

# # Happy Emoticons
# emoticons_happy = set([
#     ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
#     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
#     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
#     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
#     '<3'
#     ])
 
# # Sad Emoticons
# emoticons_sad = set([
#     ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
#     ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
#     ':c', ':{', '>:\\', ';('
#     ])
 
# # all emoticons (happy + sad)
# emoticons = emoticons_happy.union(emoticons_sad)
 
# def clean_tweets(tweet):
#     # remove stock market tickers like $GE
#     tweet = re.sub(r'\$\w*', '', tweet)
 
#     # remove old style retweet text "RT"
#     tweet = re.sub(r'^RT[\s]+', '', tweet)
 
#     # remove hyperlinks
#     tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
#     # remove hashtags
#     # only removing the hash # sign from the word
#     tweet = re.sub(r'#', '', tweet)
    
#     tweet = tweet.lower()
    
#     #remove coma
#     tweet = re.sub(r',','',tweet)
    
#     #remove angka
#     tweet = re.sub('[0-9]+', '', tweet)
 
#     # tokenize tweets
#     tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#     tweet_tokens = tokenizer.tokenize(tweet)
 
#     tweets_clean = []    
#     for word in tweet_tokens:
#         if (word not in stopwords_indonesia and # remove stopwords
#             word not in emoticons and # remove emoticons
#             word not in string.punctuation): # remove punctuation
#             stem_word = stemmer.stem(word) # stemming word
#             tweets_clean.append(word)
#     return tweets_clean

# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    tweet = tweet.lower()
    
    #remove coma
    tweet = re.sub(r',','',tweet)
    
    #remove angka
    tweet = re.sub('[0-9]+', '', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = [] 
    for word in tweet_tokens:
        if (word not in stopwords_indonesia and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
 
    return tweets_clean



tweets['Text_bersih'] = tweets['Text'].apply(lambda x: remove_punct(clean_tweets(x)))

NameError: name 'remove_punct' is not defined

In [17]:
tweets

,Text,Labels,Text_bersih
0,memang biznet lebih aman dan nyaman. internet ...,positif,biznet aman nyaman internet kenceng stabil har...
1,@biznethome halo.. biznet jogja kenapa jadi se...,negatif,halo biznet jogja ganggu ya hari kali mati los...
2,Biznet home lama-lama gue kepret ya. Sebulan 3...,negatif,biznet home lama gue kepret ya bulan kali mati...
3,@_happymeals_ @firstmediacares pakai biznet aj...,positif,pakai biznet aja mo stabil mursidah
4,"@BiznetNetworks begini nih, lagi penting mau m...",NaN,nih meeting online beda biznet mati putus suli...


In [9]:
def remove_punct(text):
    text  = " ".join([char for char in text if char not in string.punctuation])
    return text

tweets['clean'] = tweets['Text_bersih'].apply(lambda x: remove_punct(clean_tweets(x)))

tweets.sort_values("Text_bersih", inplace = True)
tweets.drop_duplicates(subset ="Text_bersih", keep = 'first', inplace = True)

In [72]:
tweets = pd.read_csv('../Dataset/dataset_clear.csv')


In [78]:
# Cek total dari label positif dan negatif

tweets['Labels'].value_counts()

positif    361
negatif    267
Name: Labels, dtype: int64

In [19]:
# Melabelkan untuk Star 5 label 1, selain 5 label 0

label = []
for index, row in tweets.iterrows():
    if row["Labels"] == 'positif':
        label.append(1)
    else:
        label.append(0)
        
# Menambahkan kolom labels dari frame label
tweets["Labels"] = label

# Menyamakan data label 1 dan 0 menjadi 25000 sample
s_1 = tweets[tweets['Labels']==1].sample(250,replace=True)
s_2 = tweets[tweets['Labels']==0].sample(250,replace=True)
tweets = pd.concat([s_1, s_2])

# menampilkan total data dan total colom, menampilkan perbandingan dari kolom label
print(tweets.shape)
print(tweets['Labels'].value_counts(normalize=True))

(500, 2)
1    0.5
0    0.5
Name: Labels, dtype: float64


In [20]:
# Membagi data train dan data test

# membagi data test size dengan ukuran 0.1
X_train, X_test, y_train, y_test = train_test_split(tweets['Text'], tweets['Labels'], 
                                                    test_size=0.5, stratify=tweets['Labels'], random_state=30)

In [21]:
# Implementasi TF IDF pada dokumen kita

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

print(X_train)
# print(X_test)

  (0, 222)	0.6391994679154291
  (0, 729)	0.4407756689836589
  (0, 78)	0.13531015700243748
  (0, 416)	0.6154933072427483
  (1, 428)	0.6192099701392372
  (1, 526)	0.521928315128477
  (1, 467)	0.5761621520245644
  (1, 78)	0.11048538963019872
  (2, 423)	0.542355589060548
  (2, 54)	0.5316115938773294
  (2, 325)	0.6353955446748919
  (2, 78)	0.13968546838268134
  (3, 606)	0.2818039724342164
  (3, 21)	0.2375308532193518
  (3, 401)	0.2483126506398665
  (3, 60)	0.22872148329058659
  (3, 530)	0.2818039724342164
  (3, 172)	0.15976641221013724
  (3, 290)	0.2818039724342164
  (3, 402)	0.2375308532193518
  (3, 260)	0.18133000705116661
  (3, 550)	0.2622128050849365
  (3, 581)	0.2622128050849365
  (3, 681)	0.22872148329058659
  (3, 647)	0.2818039724342164
  :	:
  (246, 78)	0.05841958093059371
  (247, 483)	0.3902857748700633
  (247, 280)	0.3902857748700633
  (247, 124)	0.3902857748700633
  (247, 309)	0.3631528928759754
  (247, 657)	0.3631528928759754
  (247, 117)	0.26006951361946257
  (247, 94)	0.343901

In [30]:
print(X_train.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
 X_train.get_feature_names() 

AttributeError: get_feature_names not found

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(X_train)
# df_idf = pd.DataFrame(tfidf_transformer.idf_, index=tweets.get_feature_names(),columns=["idf_weights"])
def word_count(sent):
    word2cnt = dict()
    for word in sent.split():
        if word in word2cnt: word2cnt[word] += 1
        else: word2cnt[word] = 1
    return word2cnt

tweets['idf_weights'] = tweets['Text'].apply(word_count)
tweets
# tweets.sort_values(by=['idf_weights']).head(40)

,Text,Labels,idf_weights
359,better biznet kah hargae gak kak,1,"{'better': 1, 'biznet': 1, 'kah': 1, 'hargae':..."
442,ganti atuh ku biznet lamun geus aya,1,"{'ganti': 1, 'atuh': 1, 'ku': 1, 'biznet': 1, ..."
495,wkwkwkwk maen gua kasih tau kuat biznet,1,"{'wkwkwkwk': 1, 'maen': 1, 'gua': 1, 'kasih': ..."
351,biznet my republic by u first media msh bnyk l...,1,"{'biznet': 1, 'my': 1, 'republic': 1, 'by': 1,..."
418,biznet ok ga sih,1,"{'biznet': 1, 'ok': 1, 'ga': 1, 'sih': 1}"
...,...,...,...
146,jam indihomo ngajak antem kalo biznet bantul p...,0,"{'jam': 1, 'indihomo': 1, 'ngajak': 1, 'antem'..."
204,halo biznet ganggu jaring area budur sidoarjo ...,0,"{'halo': 1, 'biznet': 1, 'ganggu': 1, 'jaring'..."
179,bejat malang raya kyk e pindah biznet ae gass,0,"{'bejat': 1, 'malang': 1, 'raya': 1, 'kyk': 1,..."
67,biznet ngehe banget jaringanya,0,"{'biznet': 1, 'ngehe': 1, 'banget': 1, 'jaring..."


In [28]:
tf_idf_vector=tfidf_transformer.transform(X_train)
feature_names = tweets['Text']


first_document_vector=tf_idf_vector[1]
#df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=['tfidf'])
def word_count(sent):
    word2cnt = dict()
    for word in sent.split():
        if word in word2cnt: word2cnt[word] += 1
        else: word2cnt[word] = 1
    return word2cnt
tweets['tfidf'] = tweets['Text'].apply(word_count)
tweets

#df.sort_values(by=['tfidf'],ascending=False).head(45)

,Text,Labels,idf_weights,tfidf
359,better biznet kah hargae gak kak,1,"{'better': 1, 'biznet': 1, 'kah': 1, 'hargae':...","{'better': 1, 'biznet': 1, 'kah': 1, 'hargae':..."
442,ganti atuh ku biznet lamun geus aya,1,"{'ganti': 1, 'atuh': 1, 'ku': 1, 'biznet': 1, ...","{'ganti': 1, 'atuh': 1, 'ku': 1, 'biznet': 1, ..."
495,wkwkwkwk maen gua kasih tau kuat biznet,1,"{'wkwkwkwk': 1, 'maen': 1, 'gua': 1, 'kasih': ...","{'wkwkwkwk': 1, 'maen': 1, 'gua': 1, 'kasih': ..."
351,biznet my republic by u first media msh bnyk l...,1,"{'biznet': 1, 'my': 1, 'republic': 1, 'by': 1,...","{'biznet': 1, 'my': 1, 'republic': 1, 'by': 1,..."
418,biznet ok ga sih,1,"{'biznet': 1, 'ok': 1, 'ga': 1, 'sih': 1}","{'biznet': 1, 'ok': 1, 'ga': 1, 'sih': 1}"
...,...,...,...,...
146,jam indihomo ngajak antem kalo biznet bantul p...,0,"{'jam': 1, 'indihomo': 1, 'ngajak': 1, 'antem'...","{'jam': 1, 'indihomo': 1, 'ngajak': 1, 'antem'..."
204,halo biznet ganggu jaring area budur sidoarjo ...,0,"{'halo': 1, 'biznet': 1, 'ganggu': 1, 'jaring'...","{'halo': 1, 'biznet': 1, 'ganggu': 1, 'jaring'..."
179,bejat malang raya kyk e pindah biznet ae gass,0,"{'bejat': 1, 'malang': 1, 'raya': 1, 'kyk': 1,...","{'bejat': 1, 'malang': 1, 'raya': 1, 'kyk': 1,..."
67,biznet ngehe banget jaringanya,0,"{'biznet': 1, 'ngehe': 1, 'banget': 1, 'jaring...","{'biznet': 1, 'ngehe': 1, 'banget': 1, 'jaring..."


In [29]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(tweets)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[1]
# df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=['tfidf'])
# df.sort_values(by=['tfidf'],ascending=False).head(45)

def word_count(sent):
    word2cnt = dict()
    for word in sent.split():
        if word in word2cnt: word2cnt[word] += 1
        else: word2cnt[word] = 1
    return word2cnt
tweets['tfidf_1'] = tweets['Text'].apply(word_count)
tweets

,Text,Labels,idf_weights,tfidf,tfidf_1
359,better biznet kah hargae gak kak,1,"{'better': 1, 'biznet': 1, 'kah': 1, 'hargae':...","{'better': 1, 'biznet': 1, 'kah': 1, 'hargae':...","{'better': 1, 'biznet': 1, 'kah': 1, 'hargae':..."
442,ganti atuh ku biznet lamun geus aya,1,"{'ganti': 1, 'atuh': 1, 'ku': 1, 'biznet': 1, ...","{'ganti': 1, 'atuh': 1, 'ku': 1, 'biznet': 1, ...","{'ganti': 1, 'atuh': 1, 'ku': 1, 'biznet': 1, ..."
495,wkwkwkwk maen gua kasih tau kuat biznet,1,"{'wkwkwkwk': 1, 'maen': 1, 'gua': 1, 'kasih': ...","{'wkwkwkwk': 1, 'maen': 1, 'gua': 1, 'kasih': ...","{'wkwkwkwk': 1, 'maen': 1, 'gua': 1, 'kasih': ..."
351,biznet my republic by u first media msh bnyk l...,1,"{'biznet': 1, 'my': 1, 'republic': 1, 'by': 1,...","{'biznet': 1, 'my': 1, 'republic': 1, 'by': 1,...","{'biznet': 1, 'my': 1, 'republic': 1, 'by': 1,..."
418,biznet ok ga sih,1,"{'biznet': 1, 'ok': 1, 'ga': 1, 'sih': 1}","{'biznet': 1, 'ok': 1, 'ga': 1, 'sih': 1}","{'biznet': 1, 'ok': 1, 'ga': 1, 'sih': 1}"
...,...,...,...,...,...
146,jam indihomo ngajak antem kalo biznet bantul p...,0,"{'jam': 1, 'indihomo': 1, 'ngajak': 1, 'antem'...","{'jam': 1, 'indihomo': 1, 'ngajak': 1, 'antem'...","{'jam': 1, 'indihomo': 1, 'ngajak': 1, 'antem'..."
204,halo biznet ganggu jaring area budur sidoarjo ...,0,"{'halo': 1, 'biznet': 1, 'ganggu': 1, 'jaring'...","{'halo': 1, 'biznet': 1, 'ganggu': 1, 'jaring'...","{'halo': 1, 'biznet': 1, 'ganggu': 1, 'jaring'..."
179,bejat malang raya kyk e pindah biznet ae gass,0,"{'bejat': 1, 'malang': 1, 'raya': 1, 'kyk': 1,...","{'bejat': 1, 'malang': 1, 'raya': 1, 'kyk': 1,...","{'bejat': 1, 'malang': 1, 'raya': 1, 'kyk': 1,..."
67,biznet ngehe banget jaringanya,0,"{'biznet': 1, 'ngehe': 1, 'banget': 1, 'jaring...","{'biznet': 1, 'ngehe': 1, 'banget': 1, 'jaring...","{'biznet': 1, 'ngehe': 1, 'banget': 1, 'jaring..."


In [28]:
def preprocess_data(data):
    
    # funtion clean data
    #data = clean_tweets(data)
    
    # function hapus tanda baca
    #data = remove_punct(data)
    
    # function TF-IDF
    data = vectorizer.transform([data])
    
    return data

In [16]:
# Klasifikasi svm dengan kernel linear

clf = svm.SVC(kernel="linear")

#cross_val_score(clf, X_train, y_train, cv=10)

In [30]:
# lakukan prediksi SVM pada data test

clf.fit(X_train,y_train)
predict = clf.predict(X_test)

In [31]:
import pickle
pickle.dump(clf, open('model.model','wb'))
model = pickle.load(open('model.model','rb'))


In [32]:
# f1_score
print("f1 score hasil prediksi adalah: ")
print(f1_score(y_test, predict))

# accuracy score
print("accuracy score hasil prediksi adalah: ")
print(accuracy_score(y_test, predict))

# precision score
print("precision score hasil prediksi adalah: ")
print(precision_score(y_test, predict))

# recall score
print("recall score hasil prediksi adalah: ")
print(recall_score(y_test, predict))

f1 score hasil prediksi adalah: 
1.0
accuracy score hasil prediksi adalah: 
1.0
precision score hasil prediksi adalah: 
1.0
recall score hasil prediksi adalah: 
1.0


In [22]:
# Confusion matrix

tn, fp, fn, tp = confusion_matrix(y_test, predict).ravel()
tn, fp, fn, tp

(1, 0, 0, 1)

In [23]:
# Contoh kalimat 

review_positif = "ganti biznet aja dari pada indihome....."
review_negatif = "biznet masalah mulu"

In [29]:
# cek prediksi dari kalimat
model.predict(preprocess_data(review_positif))
#clf.predict

array([1], dtype=int64)

In [29]:
# cek prediksi dari kalimat
clf.predict(preprocess_data(review_negatif))

array([0], dtype=int64)